Ön hazırlık (kütüphane kurulumu)

In [1]:
# Gerekli kütüphaneleri yükleyelim (Colab hücresine)
!pip install cirq qiskit qutip numpy matplotlib --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.8/670.8 kB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 430.5/430.5 kB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 92.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.8/31.8 MB 54.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 83.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 87.2 MB/s eta 0:00:00


QUANTUM K-MEANS

1. Cirq ile Quantum k-Means (Swap Test ile Mesafe Hesabı)

In [2]:
import cirq
import numpy as np

# Qubitler: kontrol, x_i, c_j
control = cirq.GridQubit(0, 0)
x_qubit = cirq.GridQubit(0, 1)
c_qubit = cirq.GridQubit(0, 2)

# Swap Test devresi
def swap_test():
    circuit = cirq.Circuit()
    circuit.append(cirq.H(control))
    circuit.append(cirq.CSWAP(control, x_qubit, c_qubit))
    circuit.append(cirq.H(control))
    circuit.append(cirq.measure(control, key='result'))
    return circuit

circuit = swap_test()

print("=== Cirq Quantum k-Means (Swap Test) ===")
print(circuit)

# Simülasyon
sim = cirq.Simulator()
result = sim.run(circuit, repetitions=100)
print("\nMeasurement results:", result.histogram(key='result'))


=== Cirq Quantum k-Means (Swap Test) ===
(0, 0): ───H───@───H───M('result')───
               │
(0, 1): ───────×─────────────────────
               │
(0, 2): ───────×─────────────────────

Measurement results: Counter({0: 100})


2. Qiskit ile Quantum k-Means (Swap Test)

In [3]:
from qiskit import QuantumCircuit, Aer, execute
from qiskit.circuit.library import HGate, CSXGate
import numpy as np

qc = QuantumCircuit(3, 1)  # kontrol + iki veri qubiti
qc.h(0)
qc.cswap(0, 1, 2)
qc.h(0)
qc.measure(0, 0)

backend = Aer.get_backend('qasm_simulator')
result = execute(qc, backend, shots=1024).result()
counts = result.get_counts()

print("=== Qiskit Quantum k-Means (Swap Test) ===")
print(qc)
print("\nMeasurement counts:", counts)


ImportError: cannot import name 'Aer' from 'qiskit' (/usr/local/lib/python3.12/dist-packages/qiskit/__init__.py)

3. QuTiP ile Quantum k-Means (Kuantum Benzerlik Hesabı)

In [4]:
from qutip import basis, tensor, hadamard_transform, cnot, swap
import numpy as np

# Qubit durumları
control = basis(2, 0)
x_state = basis(2, 0)
c_state = basis(2, 1)

# Başlangıç durumu
psi0 = tensor(control, x_state, c_state)

# H kontrol qubitine
H = tensor(hadamard_transform(1), qeye(2), qeye(2))
psi1 = H * psi0

# Kontrollü SWAP işlemi (swap test)
SWAP = swap()
psi2 = SWAP * psi1

# Son H ve ölçüm
psi_final = H * psi2

print("=== QuTiP Quantum k-Means (Swap Test) ===")
print(psi_final)


ImportError: cannot import name 'hadamard_transform' from 'qutip' (/usr/local/lib/python3.12/dist-packages/qutip/__init__.py)

QUANTUM SUPPORT VECTOR MACHINE (QSVM)

1. Cirq ile Quantum Kernel Hesaplaması (Swap Test)

In [5]:
import cirq
import numpy as np

# Örnek veriler
x0 = np.array([1, 0])
x1 = np.array([0, 1])

def encode_state(vec):
    norm = np.linalg.norm(vec)
    return [vec[0]/norm, vec[1]/norm]

x0_state = encode_state(x0)
x1_state = encode_state(x1)

# Qubitler: kontrol, x, y
control = cirq.GridQubit(0, 0)
x_qubit = cirq.GridQubit(0, 1)
y_qubit = cirq.GridQubit(0, 2)

# Başlangıç durumu
circuit = cirq.Circuit()
circuit.append(cirq.ry(2*np.arccos(x0_state[0]))(x_qubit))
circuit.append(cirq.ry(2*np.arccos(x1_state[0]))(y_qubit))

# Swap Test
circuit.append(cirq.H(control))
circuit.append(cirq.CSWAP(control, x_qubit, y_qubit))
circuit.append(cirq.H(control))
circuit.append(cirq.measure(control, key='result'))

sim = cirq.Simulator()
result = sim.run(circuit, repetitions=1000)
prob_zero = result.histogram(key='result').get(0,0)/1000
kernel = 2*prob_zero - 1  # swap test formula

print("Quantum Kernel K(x0,x1) ≈", kernel)


Quantum Kernel K(x0,x1) ≈ 0.0020000000000000018


2. Qiskit ile Swap Test (QSVM)

In [6]:
from qiskit import QuantumCircuit, Aer, execute
import numpy as np

qc = QuantumCircuit(3,1)

# Örnek veriler
theta0 = 0  # x0 kodlama
theta1 = np.pi/2  # x1 kodlama

qc.ry(2*theta0,1)
qc.ry(2*theta1,2)

# Swap Test
qc.h(0)
qc.cswap(0,1,2)
qc.h(0)
qc.measure(0,0)

backend = Aer.get_backend('qasm_simulator')
counts = execute(qc, backend, shots=1024).result().get_counts()
prob_zero = counts.get('0',0)/1024
kernel = 2*prob_zero - 1

print("Quantum Kernel K(x0,x1) ≈", kernel)


ImportError: cannot import name 'Aer' from 'qiskit' (/usr/local/lib/python3.12/dist-packages/qiskit/__init__.py)

3. QuTiP ile Quantum Kernel

In [7]:
from qutip import basis, tensor, hadamard_transform, cnot, swap
import numpy as np

# Kuantum durumları
x0 = basis(2,0)
x1 = basis(2,1)
control = basis(2,0)

# Başlangıç durumu
psi0 = tensor(control, x0, x1)

# H kontrol qubitine
H = tensor(hadamard_transform(1), qeye(2), qeye(2))
psi1 = H * psi0

# Swap Test
SWAP = swap()
psi2 = SWAP * psi1
psi_final = H * psi2

print("QSVM Swap Test final state:\n", psi_final)


ImportError: cannot import name 'hadamard_transform' from 'qutip' (/usr/local/lib/python3.12/dist-packages/qutip/__init__.py)

QUANTUM NEURAL NETWORKS (QNN)

Cirq Örneği

In [8]:
import cirq
import numpy as np
from scipy.optimize import minimize

# Qubitler
qubits = [cirq.GridQubit(0, i) for i in range(2)]

# Parametrik devre tanımı
def create_circuit(params):
    circuit = cirq.Circuit()
    circuit.append(cirq.rx(params[0])(qubits[0]))
    circuit.append(cirq.ry(params[1])(qubits[1]))
    circuit.append(cirq.CNOT(qubits[0], qubits[1]))
    return circuit

# Ölçüm ve beklenen değer fonksiyonu
def expectation(params):
    simulator = cirq.Simulator()
    circuit = create_circuit(params)
    circuit.append(cirq.measure(*qubits, key='m'))
    result = simulator.simulate(circuit)
    # Örnek basit loss: |<00|psi>|^2
    psi = result.final_state_vector
    return 1 - np.abs(psi[0])**2

# Başlangıç parametreleri
params0 = np.random.rand(2)
res = minimize(expectation, params0, method='COBYLA')
print("Optimal parametreler:", res.x)


Optimal parametreler: [0.98025531 0.34714735]


Qiskit Örneği

In [9]:
from qiskit import QuantumCircuit, Aer, execute
from qiskit.circuit import Parameter
from scipy.optimize import minimize
import numpy as np

# Parametreler
theta1 = Parameter('θ1')
theta2 = Parameter('θ2')

# Devre tanımı
qc = QuantumCircuit(2)
qc.rx(theta1, 0)
qc.ry(theta2, 1)
qc.cx(0, 1)

# Ölçüm için backend
backend = Aer.get_backend('statevector_simulator')

# Beklenen değer fonksiyonu
def expectation(params):
    bound_qc = qc.bind_parameters({theta1: params[0], theta2: params[1]})
    result = execute(bound_qc, backend).result()
    psi = result.get_statevector()
    return 1 - np.abs(psi[0])**2  # basit loss

params0 = np.random.rand(2)
res = minimize(expectation, params0, method='COBYLA')
print("Optimal parametreler:", res.x)


ImportError: cannot import name 'Aer' from 'qiskit' (/usr/local/lib/python3.12/dist-packages/qiskit/__init__.py)

QuTiP Örneği

In [10]:
import qutip as qt
import numpy as np
from scipy.optimize import minimize

# Başlangıç durumu |00>
psi0 = qt.tensor(qt.basis(2,0), qt.basis(2,0))

# Parametrik devre operatörü
def U(params):
    rx1 = qt.rx(params[0])
    ry2 = qt.ry(params[1])
    cnot = qt.cnot()
    return cnot * qt.tensor(rx1, ry2)

# Beklenen değer
def expectation(params):
    psi = U(params) * psi0
    return 1 - abs((qt.tensor(qt.basis(2,0), qt.basis(2,0)).dag() * psi)[0,0])**2

params0 = np.random.rand(2)
res = minimize(expectation, params0, method='COBYLA')
print("Optimal parametreler:", res.x)


AttributeError: module 'qutip' has no attribute 'rx'

QUANTUM BOLTZMANN MACHINES

Cirq ile Basit QBM

In [11]:
import cirq
import numpy as np

# 2 qubit oluştur
qubits = [cirq.GridQubit(0, i) for i in range(2)]

# Parametreler (bias ve ağırlık)
a = [0.5, -0.3]   # Z bias
b = [0.4, 0.2]    # X tünelleme
W = [[0, 0.6], [0.6, 0]]  # Z-Z etkileşimi

# Hamiltonyen benzeri parametrik devre
circuit = cirq.Circuit()
circuit.append(cirq.rz(2 * a[0])(qubits[0]))
circuit.append(cirq.rz(2 * a[1])(qubits[1]))
circuit.append(cirq.rx(2 * b[0])(qubits[0]))
circuit.append(cirq.rx(2 * b[1])(qubits[1]))
circuit.append(cirq.CNOT(qubits[0], qubits[1]))
circuit.append(cirq.rz(2 * W[0][1])(qubits[1]))

# Ölçüm
circuit.append(cirq.measure(*qubits, key='m'))

simulator = cirq.Simulator()
result = simulator.run(circuit, repetitions=1000)
print(result.histogram(key='m'))


Counter({0: 814, 3: 148, 1: 34, 2: 4})


Qiskit ile Basit QBM

In [12]:
from qiskit import QuantumCircuit, Aer, execute
import numpy as np

qc = QuantumCircuit(2)
a = [0.5, -0.3]
b = [0.4, 0.2]
W = [[0, 0.6], [0.6, 0]]

qc.rz(2 * a[0], 0)
qc.rz(2 * a[1], 1)
qc.rx(2 * b[0], 0)
qc.rx(2 * b[1], 1)
qc.cx(0,1)
qc.rz(2 * W[0][1],1)
qc.measure_all()

backend = Aer.get_backend('qasm_simulator')
result = execute(qc, backend, shots=1000).result()
counts = result.get_counts()
print(counts)


ImportError: cannot import name 'Aer' from 'qiskit' (/usr/local/lib/python3.12/dist-packages/qiskit/__init__.py)

QuTiP ile Basit QBM

In [13]:
import qutip as qt
import numpy as np

# Pauli matrisleri
I = qt.qeye(2)
X = qt.sigmax()
Z = qt.sigmaz()

# Parametreler
a = [0.5, -0.3]
b = [0.4, 0.2]
W = [[0, 0.6], [0.6, 0]]

# Hamiltonyen oluşturma
H = -a[0]*qt.tensor(Z,I) - a[1]*qt.tensor(I,Z) - W[0][1]*qt.tensor(Z,Z) - b[0]*qt.tensor(X,I) - b[1]*qt.tensor(I,X)

# Termal Gibbs durumu
beta = 1.0
rho = (-beta * H).expm()
rho = rho / rho.tr()  # normalize

# Olasılıkları yazdır
for i in range(4):
    state = qt.basis(4,i)
    print(f"|{i:02b}> probability:", (state.dag()*rho*state).full()[0,0].real)


TypeError: incompatible dimensions [[1], [4]] and [[2, 2], [2, 2]]

QUANTUM KERNEL METHODS

Cirq ile Quantum Kernel Örneği

In [14]:
import cirq
import numpy as np

# Qubitler
q = cirq.GridQubit.rect(1,1)

# İki veri noktası
x = np.array([0.2, 0.8])
z = np.array([0.5, 0.3])

# Kuantum durumu hazırlama fonksiyonu (Angle Encoding)
def prepare_state(qubit, data):
    circuit = cirq.Circuit()
    circuit.append(cirq.rx(data[0]*np.pi)(qubit))
    circuit.append(cirq.ry(data[1]*np.pi)(qubit))
    return circuit

# Swap test ile iç-çarpım
def swap_test(circuit_x, circuit_z):
    ancilla = cirq.GridQubit(1,0)
    c = cirq.Circuit()
    c.append(cirq.H(ancilla))
    c.append(cirq.CNOT(ancilla,q))
    c.append(cirq.CNOT(ancilla,q))
    c.append(cirq.H(ancilla))
    c.append(cirq.measure(ancilla, key='m'))
    return c

# Basit örnek için iç-çarpımı yaklaşık sayalım
print("Cirq swap test örneği, Circuit hazırlığı gösterildi.")


Cirq swap test örneği, Circuit hazırlığı gösterildi.


Qiskit ile Quantum Kernel Örneği

In [15]:
from qiskit import QuantumCircuit, Aer, execute
import numpy as np

x = np.array([0.2,0.8])
z = np.array([0.5,0.3])

def feature_map(data):
    qc = QuantumCircuit(1)
    qc.rx(np.pi*data[0],0)
    qc.ry(np.pi*data[1],0)
    return qc

qc_x = feature_map(x)
qc_z = feature_map(z)

# Kuantum kernel: iç-çarpım |<x|z>|^2
backend = Aer.get_backend('statevector_simulator')
sv_x = execute(qc_x, backend).result().get_statevector()
sv_z = execute(qc_z, backend).result().get_statevector()
kernel = np.abs(np.vdot(sv_x, sv_z))**2
print("Kernel K(x,z) =", kernel)


ImportError: cannot import name 'Aer' from 'qiskit' (/usr/local/lib/python3.12/dist-packages/qiskit/__init__.py)

QuTiP ile Quantum Kernel Örneği

In [16]:
import qutip as qt
import numpy as np

# 1-qubit durumları
def angle_encoding(data):
    rx = qt.rx(np.pi*data[0])
    ry = qt.ry(np.pi*data[1])
    return ry * rx * qt.basis(2,0)

x = np.array([0.2,0.8])
z = np.array([0.5,0.3])

state_x = angle_encoding(x)
state_z = angle_encoding(z)

kernel = np.abs((state_z.dag() * state_x)[0,0])**2
print("QuTiP kernel K(x,z) =", kernel)


AttributeError: module 'qutip' has no attribute 'rx'